In [ ]:
pip install pytorch-tabnet pandas scikit-learn numpy matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 890.3 kB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!ls -l '/content/gdrive/'

total 4
drwx------ 2 root root 4096 Nov 30 15:42 MyDrive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных
df = pd.read_csv('/content/gdrive/MyDrive/all_auto_ru_09_09_2020.csv')

# Предварительный анализ данных
print(f"Размер датасета: {df.shape}")
print("\nПервые 5 строк:")
print(df.head())
print("\nИнформация о столбцах:")
print(df.info())
print("\nПропущенные значения:")
print(df.isnull().sum())

Размер датасета: (89378, 26)

Первые 5 строк:
          bodyType brand   color fuelType  modelDate                   name  \
0            Седан  AUDI  040001   бензин     1990.0  2.8 MT (174 л.с.) 4WD   
1            Седан  AUDI  EE1D19   бензин     1982.0       1.8 MT (90 л.с.)   
2  Универсал 5 дв.  AUDI  0000CC   бензин     1988.0  2.3 MT (136 л.с.) 4WD   
3            Седан  AUDI  CACECB   бензин     1988.0       1.8 MT (90 л.с.)   
4            Седан  AUDI  040001   бензин     1990.0      2.0 MT (101 л.с.)   

   numberOfDoors  productionDate          vehicleConfiguration  \
0            4.0            1991          SEDAN MECHANICAL 2.8   
1            4.0            1986          SEDAN MECHANICAL 1.8   
2            5.0            1989  WAGON_5_DOORS MECHANICAL 2.3   
3            4.0            1989          SEDAN MECHANICAL 1.8   
4            4.0            1991          SEDAN MECHANICAL 2.0   

  vehicleTransmission  ...  Руль  Состояние Владельцы        ПТС Таможня  \
0     

In [ ]:
# Выберем наиболее релевантные признаки для прогнозирования цены
selected_features = ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name', 'numberOfDoors', 'vehicleConfiguration',
                      'vehicleTransmission', 'engineDisplacement', 'enginePower', 'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Владельцы',
                      'ПТС', 'Таможня', 'price', 'start_date', 'model'
                    ]

# Создаем копию датафрейма с выбранными признаками
data = df[selected_features].copy()

# Преобразование типов данных ДО обработки пропущенных значений
def convert_dtypes(data):
    # Преобразование enginePower - извлекаем числа и преобразуем в float
    if 'enginePower' in data.columns:
        data['enginePower'] = data['enginePower'].astype(str).str.extract(r'(\d+)', expand=False).astype(float)

    # Преобразование engineDisplacement - извлекаем числа с точкой
    if 'engineDisplacement' in data.columns:
        data['engineDisplacement'] = data['engineDisplacement'].astype(str).str.extract(r'(\d+\.?\d*)', expand=False).astype(float)

    # Преобразование других числовых признаков
    numeric_features = ['modelDate', 'numberOfDoors', 'mileage', 'Владельцы', 'price']
    for feature in numeric_features:
        if feature in data.columns:
            # Используем coerce для преобразования проблемных значений в NaN
            data[feature] = pd.to_numeric(data[feature], errors='coerce')

    return data

data = convert_dtypes(data)

# Обработка пропущенных значений ПОСЛЕ преобразования типов
def preprocess_data(data):
    # Удаление строк с пропущенной ценой
    data = data.dropna(subset=['price'])

    # Заполнение пропусков в числовых признаках
    numeric_features = ['modelDate', 'numberOfDoors', 'engineDisplacement', 'enginePower', 'mileage', 'Владельцы']
    for feature in numeric_features:
        if feature in data.columns:
            data.loc[:, feature] = data[feature].fillna(data[feature].median())

    # Заполнение пропусков в категориальных признаках
    categorical_features = ['bodyType', 'color', 'fuelType', 'vehicleTransmission', 'brand', 'name',
                           'vehicleConfiguration', 'Комплектация', 'Привод', 'Руль', 'ПТС', 'Таможня']
    for feature in categorical_features:
        if feature in data.columns:
            data.loc[:, feature] = data[feature].fillna('unknown')

    return data

data = preprocess_data(data)

print(f"Размер данных после очистки: {data.shape}")

Размер данных после очистки: (88968, 22)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from pytorch_tabnet.utils import define_device

# Подготовка категориальных и числовых признаков
def prepare_features(data):
    # Целевая переменная
    y = data['price'].values

    # Удаляем целевую переменную и идентификаторы
    X = data.drop('price', axis=1)

    # Разделяем признаки на категориальные и числовые
    categorical_features = []
    categorical_dims = {}

    # Кодируем категориальные признаки
    for col in X.columns:
        if X[col].dtype == 'object':
            # Для TabNet используем LabelEncoder
            label_encoder = LabelEncoder()
            X[col] = label_encoder.fit_transform(X[col].astype(str))
            categorical_features.append(col)
            categorical_dims[col] = len(label_encoder.classes_)
        else:
            # Заполняем пропуски в числовых признаках
            X[col] = X[col].fillna(X[col].median())

    # Преобразуем в numpy array
    X = X.values

    return X, y, categorical_features, categorical_dims

X, y, categorical_features, categorical_dims = prepare_features(data)

print(f"Форма X: {X.shape}, Форма y: {y.shape}")
print(f"Категориальные признаки: {categorical_features}")

Форма X: (88968, 21), Форма y: (88968,)
Категориальные признаки: ['bodyType', 'brand', 'color', 'fuelType', 'name', 'vehicleConfiguration', 'vehicleTransmission', 'description', 'Комплектация', 'Привод', 'Руль', 'ПТС', 'start_date', 'model']


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Разделение на обучающую, валидационную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# Масштабирование числовых признаков (важно для TabNet)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print(f"Размеры выборок:")
print(f"Обучающая: {X_train.shape}, {y_train.shape}")
print(f"Валидационная: {X_val.shape}, {y_val.shape}")
print(f"Тестовая: {X_test.shape}, {y_test.shape}")

Размеры выборок:
Обучающая: (56939, 21), (56939,)
Валидационная: (14235, 21), (14235,)
Тестовая: (17794, 21), (17794,)


In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Определение устройства (GPU/CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

# Предполагаем, что X_train и X_val уже определены
# Если они в формате numpy array, преобразуем обратно в DataFrame для обработки
if not isinstance(X_train, pd.DataFrame):
    # Создаем имена столбцов, если их нет
    feature_names = [f'feature_{i}' for i in range(X_train.shape[1])]
    X_train = pd.DataFrame(X_train, columns=feature_names)
    X_val = pd.DataFrame(X_val, columns=feature_names)

# Определяем категориальные признаки (замените на ваши реальные категориальные колонки)
categorical_features = ['bodyType', 'brand', 'color', 'fuelType', 'vehicleTransmission',
                       'Комплектация', 'Привод', 'Руль', 'ПТС', 'Таможня']

# Оставляем только те категориальные признаки, которые есть в данных
categorical_features = [col for col in categorical_features if col in X_train.columns]
print(f"Категориальные признаки: {categorical_features}")

# Кодируем категориальные признаки
label_encoders = {}
categorical_dims = {}

for feature in categorical_features:
    le = LabelEncoder()

    # Объединяем train и val для полного охвата категорий
    combined_data = pd.concat([X_train[feature], X_val[feature]], axis=0)
    le.fit(combined_data.astype(str))

    # Преобразуем train и val данные
    X_train[feature] = le.transform(X_train[feature].astype(str))
    X_val[feature] = X_val[feature].astype(str).map(
        lambda x: le.transform([x])[0] if x in le.classes_ else -1
    )

    categorical_dims[feature] = len(le.classes_)
    label_encoders[feature] = le

# Получаем индексы категориальных признаков
cat_idxs = [i for i, col in enumerate(X_train.columns) if col in categorical_features]
cat_dims = [categorical_dims[col] for col in categorical_features]

print(f"Индексы категориальных признаков: {cat_idxs}")
print(f"Размерности категориальных признаков: {cat_dims}")

# Преобразуем обратно в numpy arrays для TabNet
X_train_np = X_train.values.astype(float)
X_val_np = X_val.values.astype(float)

# Преобразуем целевые переменные в двумерные массивы
y_train_2d = y_train.values.reshape(-1, 1) if hasattr(y_train, 'values') else y_train.reshape(-1, 1)
y_val_2d = y_val.values.reshape(-1, 1) if hasattr(y_val, 'values') else y_val.reshape(-1, 1)

# Проверяем, что размерности совпадают
assert len(cat_idxs) == len(cat_dims), f"cat_idxs и cat_dims должны иметь одинаковую длину: {len(cat_idxs)} vs {len(cat_dims)}"

# Инициализация модели TabNet
tabnet_regressor = TabNetRegressor(
    n_d=64,                           # Размерность слоя принятия решений
    n_a=64,                           # Размерность слоя внимания
    n_steps=5,                        # Количество шагов
    gamma=1.5,                        # Коэффициент масштабирования
    cat_idxs=[i for i, col in enumerate(data.drop('price', axis=1).columns)
              if col in categorical_features],  # Индексы категориальных признаков
    cat_dims=[categorical_dims[col] for col in categorical_features],  # Размерности категориальных признаков
    cat_emb_dim=1,                    # Размерность эмбеддингов для категориальных признаков
    n_independent=2,                  # Количество независимых Gated Linear Units на шаг
    n_shared=2,                       # Количество общих Gated Linear Units на шаг
    lambda_sparse=1e-4,               # Параметр разреженности
    momentum=0.3,                     # Моментум для пакетной нормализации
    clip_value=2.0,                   # Обрезка градиентов
    optimizer_fn=torch.optim.Adam,    # Оптимизатор
    optimizer_params=dict(lr=2e-2),   # Параметры оптимизатора
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    scheduler_params={"step_size": 50, "gamma": 0.9},
    mask_type='entmax',               # Тип маскирования
    device_name=device
)

# Обучение модели
max_epochs = 200
patience = 20

tabnet_regressor.fit(
    X_train=X_train_np,  # Используем numpy array, а не DataFrame
    y_train=y_train_2d,  # Используем 2D numpy array
    eval_set=[(X_val_np, y_val_2d)],  # Используем numpy arrays
    eval_name=['val'],
    eval_metric=['rmse', 'mae'],
    max_epochs=max_epochs,
    patience=patience,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    loss_fn=torch.nn.functional.mse_loss
)

Используемое устройство: cpu
Категориальные признаки: []
Индексы категориальных признаков: []
Размерности категориальных признаков: []


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 5573702125588.322| val_rmse: 2305598.10685| val_mae: 1308266.77174|  0:00:17s
epoch 1  | loss: 5566449314056.763| val_rmse: 2302809.24388| val_mae: 1306527.88176|  0:00:31s
epoch 2  | loss: 5550721011314.116| val_rmse: 2298450.71198| val_mae: 1303394.37477|  0:00:49s
epoch 3  | loss: 5526449328689.466| val_rmse: 2292313.46726| val_mae: 1298913.51521|  0:01:03s
epoch 4  | loss: 5500957872358.27| val_rmse: 2291234.7293| val_mae: 1297345.27285|  0:01:18s
epoch 5  | loss: 5478733177706.031| val_rmse: 2282294.64406| val_mae: 1291226.06405|  0:01:31s
epoch 6  | loss: 5456993491460.182| val_rmse: 2276388.71937| val_mae: 1286799.8946|  0:01:45s
epoch 7  | loss: 5434208753133.361| val_rmse: 2272934.50295| val_mae: 1284193.34042|  0:01:59s
epoch 8  | loss: 5406080366752.385| val_rmse: 2267486.7239| val_mae: 1278961.76351|  0:02:12s
epoch 9  | loss: 5375817871449.415| val_rmse: 2259394.05686| val_mae: 1272670.8547|  0:02:25s
epoch 10 | loss: 5343416881700.231| val_rmse: 2251742.7

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate_model(model, X_test, y_test, model_name="Model"):
    """
    Комплексная оценка метрик модели регрессии

    Parameters:
    - model: обученная модель
    - X_test: тестовые признаки
    - y_test: тестовые целевые значения
    - model_name: название модели для вывода
    - plot_results: строить ли графики
    """

    # Предсказания
    y_pred = model.predict(X_test)

    # Если модель возвращает 2D массив, преобразуем в 1D
    if y_pred.ndim > 1:
        y_pred = y_pred.flatten()
    if y_test.ndim > 1:
        y_test = y_test.flatten()

    # Расчет метрик
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Процентные ошибки
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mdape = np.median(np.abs((y_test - y_pred) / y_test)) * 100  # Медианная ошибка

    # Дополнительные метрики
    max_error = np.max(np.abs(y_test - y_pred))
    mean_error = np.mean(y_test - y_pred)
    std_error = np.std(y_test - y_pred)

    # Вывод метрик
    print("=" * 70)
    print(f"МЕТРИКИ {model_name.upper()}")
    print("=" * 70)
    print(f"MAE (Средняя абсолютная ошибка): {mae:,.2f} руб.")
    'print(f"MSE (Средняя квадратичная ошибка): {mse:,.2f}")'
    print(f"RMSE (Среднеквадратичная ошибка): {rmse:,.2f} руб.")
    print(f"R² (Коэффициент детерминации): {r2:.4f}")
    print(f"MAPE (Средняя абсолютная процентная ошибка): {mape:.2f}%")
    print(f"MdAPE (Медианная абсолютная процентная ошибка): {mdape:.2f}%")
    'print(f"Максимальная ошибка: {max_error:,.2f} руб.")'
    print(f"Средняя ошибка (смещение): {mean_error:,.2f} руб.")
    print(f"Стандартное отклонение ошибки: {std_error:,.2f} руб.")

    # Анализ распределения ошибок
    errors = y_test - y_pred
    error_stats = {
        'Средняя цена': np.mean(y_test),
        'Медианная цена': np.median(y_test),
        'MAE/Средняя цена': (mae / np.mean(y_test)) * 100,
        'Процент ошибок < 10%': np.mean(np.abs(errors / y_test) < 0.1) * 100,
        'Процент ошибок < 20%': np.mean(np.abs(errors / y_test) < 0.2) * 100,
        'Процент ошибок < 30%': np.mean(np.abs(errors / y_test) < 0.3) * 100
    }

    print("\n" + "-" * 70)
    print("АНАЛИЗ ТОЧНОСТИ:")
    print("-" * 70)
    print(f"Средняя цена в тестовой выборке: {error_stats['Средняя цена']:,.2f} руб.")
    print(f"Медианная цена в тестовой выборке: {error_stats['Медианная цена']:,.2f} руб.")
    print(f"MAE составляет {error_stats['MAE/Средняя цена']:.2f}% от средней цены")
    print(f"Ошибка < 10%: {error_stats['Процент ошибок < 10%']:.2f}% наблюдений")
    print(f"Ошибка < 20%: {error_stats['Процент ошибок < 20%']:.2f}% наблюдений")
    print(f"Ошибка < 30%: {error_stats['Процент ошибок < 30%']:.2f}% наблюдений")

    return {
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'r2': r2,
        'mape': mape,
        'mdape': mdape,
        'max_error': max_error,
        'mean_error': mean_error,
        'std_error': std_error,
        'error_stats': error_stats
    }

metrics = evaluate_model(tabnet_regressor, X_val_np, y_val_2d, "TabNet Auto Prices")


NameError: name 'tabnet_regressor' is not defined

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Предсказания на тестовой выборке
y_pred = tabnet_regressor.predict(X_test)

# Метрики
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    # Средняя абсолютная процентная ошибка (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return mae, mse, rmse, r2, mape

mae, mse, rmse, r2, mape = calculate_metrics(y_test, y_pred)

print("\n" + "="*50)
print("РЕЗУЛЬТАТЫ TABNET")
print("="*50)
print(f"MAE (Средняя абсолютная ошибка): {mae:.2f} руб.")
print(f"MSE (Средняя квадратичная ошибка): {mse:.2f}")
print(f"RMSE (Среднеквадратичная ошибка): {rmse:.2f} руб.")
print(f"R² (Коэффициент детерминации): {r2:.4f}")
print(f"MAPE (Средняя абсолютная процентная ошибка): {mape:.2f}%")


РЕЗУЛЬТАТЫ TABNET
MAE (Средняя абсолютная ошибка): 184489.81 руб.
MSE (Средняя квадратичная ошибка): 506835383293.66
RMSE (Среднеквадратичная ошибка): 711923.72 руб.
R² (Коэффициент детерминации): 0.8616
MAPE (Средняя абсолютная процентная ошибка): 235.50%
